In [2]:
county_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance'] #e.g. "XX county tax"
state_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance', 'Business plan templates', 'Small business ideas', 'How to start a small business', 'office space for rent', 'website hosting service', 'freelancing', 'side hustle', 'ecommerce'] #e.g. "tax"

In [190]:
import numpy as np

In [3]:
from pytrends.request import TrendReq
from pathlib import Path
import pandas as pd
import math
import time

cfips = pd.read_csv(r'C:\Users\sbsla\OneDrive\Desktop\Stat_222_Proj\DATA\cfips_to_county_name.csv')

#Keywords to scrape
county_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance'] #e.g. "XX county tax"
state_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance', 'Business plan templates', 'Small business ideas', 'How to start a small business', 'office space for rent', 'website hosting service', 'freelancing', 'side hustle', 'ecommerce'] #e.g. "tax"

#Connect to google
pytrends = TrendReq(hl='en-US', tz=360)


In [5]:
#Pulls state level keywords for each state

states_output = dict()
for state in set(cfips['state']):
    num = math.ceil(len(state_level_kws)/5)
    df = pd.DataFrame()
    for i in range(0,num):
        keywords = state_level_kws[i*5:min(len(state_level_kws),i*5+5)]
        pytrends.build_payload(keywords, timeframe='today 5-y', geo=f'US-{state}')
        if i == 0:
            df = pytrends.interest_over_time()
            df.drop(columns=['isPartial'], inplace=True)
        else:
            result = pytrends.interest_over_time()
            result.drop(columns=['isPartial'], inplace=True)
            df = df.merge(result, how = 'left', on = 'date')
        time.sleep(1)
    states_output[state] = df

In [10]:
#Convert each dataframe to monthly indexing instead of weekly
for key in states_output.keys():
    states_output[key] = states_output[key].resample('M').mean()

In [15]:
#stupid helper function I wrote to get the dates column
def get_dates():
    pytrends.build_payload(['dog'], timeframe='today 5-y', geo=f'US')
    df = pytrends.interest_over_time()
    df.drop(columns = ['dog'], inplace = True)
    return df

In [107]:
#Pulls county level keywords for each county
dates = get_dates() #is the seed for successive merging
for i in range(3091,len(cfips)):
    state = cfips['state'][i]
    county_name = cfips['county_name'][i]
    num = math.ceil(len(county_level_kws)/5)
    df = pd.DataFrame()
    output = dates.copy()
    for i in range(0,num):
        keywords = county_level_kws[i*5:min(len(state_level_kws),i*5+5)]
        keywords = [county_name + ' ' + keyword for keyword in keywords]
        pytrends.build_payload(keywords, timeframe='today 5-y', geo=f'US-{state}')
        df = pytrends.interest_over_time()
        if len(df) != 0:
            df.drop(columns=['isPartial'], inplace=True)
            output = output.merge(df, how = 'left', on = 'date')
        time.sleep(1)
    counties_output[f'{county_name} {state}'] = output

In [108]:
len(list(counties_output.keys()))

3143

In [96]:
list(counties_output.keys())[3091]

'Oneida County WI'

In [28]:
cfips['county_name'][653]

'Macoupin County'

In [165]:
import dill
filename = 'globalsave.pkl'
dill.dump_session(filename)

In [111]:
cfips.head()

,cfips,county_name,state
0,1001,Autauga County,AL
1,1003,Baldwin County,AL
2,1005,Barbour County,AL
3,1007,Bibb County,AL
4,1009,Blount County,AL


In [134]:
list(counties_output.keys())[1]

'Baldwin County AL'

In [133]:
#Check that we have every county
target_names = cfips["county_name"] + " " + cfips["state"]
for name in list(counties_output.keys()):
    if name in list(target_names):
        break
    else:
        print(name)

In [141]:
#Convert each dataframe to monthly indexing instead of weekly
for key in counties_output.keys():
    counties_output[key] = counties_output[key].resample('M').mean()

In [151]:
def append_state_queries():
    keys = list(counties_output.keys())
    for i in range(1,len(keys)):
        key = keys[i]
        state_name = key[-2:]
        state_data = states_output[state_name]
        counties_output[key] = counties_output[key].merge(state_data, how = 'left', on = 'date')
        


In [152]:
append_state_queries()

In [220]:
#Some dataframes have fewer columns b/c google trends didnt have data for certain queries. This is a problem, so we need to
#go through and add columns of all NA where this occurs.
def include_missing_cols():
    counties_output_new = counties_output.copy()
    county_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance'] #e.g. "XX county tax"
    state_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance', 'Business plan templates', 'Small business ideas', 'How to start a small business', 'office space for rent', 'website hosting service', 'freelancing', 'side hustle', 'ecommerce'] #e.g. "tax"
    keys = list(counties_output.keys())
    for i in range(0,len(keys)):
        key = keys[i]
        county_name = key[:-2]
        county_col_names = [county_name + c for c in county_level_kws]
        all_col_names = county_col_names + state_level_kws
        df = pd.DataFrame(index = counties_output[key].index)
        for col in all_col_names:
            try:
                df[col] = counties_output[key][col]
            except:
                df[col] = np.nan
        counties_output_new[key] = df
    return(counties_output_new)

In [221]:
new = include_missing_cols()

In [225]:
new[list(new.keys())[1]]

,Baldwin County tax,Baldwin County loan,Baldwin County business,Baldwin County rent,Baldwin County college,Baldwin County apartments,Baldwin County office,Baldwin County insurance,tax,loan,...,office,insurance,Business plan templates,Small business ideas,How to start a small business,office space for rent,website hosting service,freelancing,side hustle,ecommerce
date,,,,,,,,,,,,,,,,,,,,,
2018-02-28,18.000000,0.0,0.00,9.00,0.0,19.0,29.00,0.00,39.000000,11.00,...,86.000000,60.000000,0.0,0.00,0.000000,16.00,0.0,0.00,0.00,0.000000
2018-03-31,18.750000,0.0,0.00,23.25,0.0,0.0,34.50,6.50,30.750000,12.25,...,84.000000,53.750000,0.0,0.00,12.750000,0.00,0.0,0.00,9.25,9.250000
2018-04-30,11.800000,0.0,8.60,5.40,0.0,14.2,13.60,14.60,30.800000,13.00,...,84.800000,55.800000,0.0,0.00,0.000000,4.00,0.0,0.00,3.00,26.800000
2018-05-31,18.500000,0.0,13.00,5.50,0.0,0.0,43.00,9.25,16.500000,13.25,...,79.500000,53.750000,0.0,0.25,7.750000,0.00,0.0,3.75,6.50,6.750000
2018-06-30,24.750000,0.0,0.00,19.75,0.0,0.0,54.25,12.25,16.750000,13.75,...,85.250000,57.000000,0.0,0.00,20.750000,7.25,0.0,7.25,3.75,41.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31,21.000000,0.0,8.80,16.00,0.0,12.4,47.40,10.80,19.000000,17.00,...,78.400000,57.600000,0.0,0.00,17.800000,12.20,0.0,0.00,17.00,24.800000
2022-11-30,9.500000,0.0,6.25,11.75,0.0,0.0,29.25,4.25,16.500000,13.75,...,69.500000,49.750000,0.0,0.00,0.000000,10.75,0.0,9.25,15.00,44.000000
2022-12-31,10.000000,0.0,3.75,11.00,0.0,0.0,36.75,31.75,19.500000,13.00,...,73.250000,52.750000,0.0,0.00,3.750000,23.50,0.0,2.75,25.50,29.000000


In [224]:
#check that all counties have the same number of columns now
for name in list(new.keys()):
    if len(new[name].columns) == 24:
        break
    else:
        print(name)

In [242]:
list_of_df = list(new.values())

In [245]:
#the first 652 frames have an extra month so we drop the first month recorded so all frames have same length
for i in range(0,653):
    list_of_df[i] = list_of_df[i].iloc[1: , :]

In [230]:
def stack_dataframes_to_3d_array(dfs): #thanks chatgpt
    arr = np.concatenate([df.values[:,:,np.newaxis] for df in dfs], axis=2)
    return(arr)

In [246]:
block = stack_dataframes_to_3d_array(list_of_df)

In [254]:
#rename everything and prepare to save
data = block

import json
import pickle

county_level_kws = ['XX county tax', 'XX county loan', 'XX county business', 'XX county rent', 'XX county college', 'XX county apartments', 'XX county office', 'XX county insurance'] #e.g. "XX county tax"
state_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance', 'Business plan templates', 'Small business ideas', 'How to start a small business', 'office space for rent', 'website hosting service', 'freelancing', 'side hustle', 'ecommerce'] #e.g. "tax"
all_kws = county_level_kws + state_level_kws
data_description = dict()
for kw in all_kws:
    data_description[kw] = "Verbatim Gtrends search query. If a county is not specified, the query is statewide."

data_name = 'Gtrends_data'

# save data in pickle format
with open(f'BLOCKED_{data_name}_data.pkl', 'wb') as f:
    pickle.dump(data, f)

# save data in json format
with open(f'BLOCKED_{data_name}_data_description.json', 'w') as f:
    json.dump(data_description, f)